<a href="https://colab.research.google.com/github/BakrAsskali/AE-VAE-GANs/blob/main/GANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

In [2]:
import sys

# Print the current system path
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize the Kaggle API
api = KaggleApi()
# If you haven't set your Kaggle API key, you can set it here:
# api.authenticate()

# Download the dataset
dataset_name = 'bryanb/abstract-art-gallery'
api.dataset_download_files(dataset_name, path='./abstract_art_gallery', unzip=True)

print("Dataset downloaded successfully.")

Dataset downloaded successfully.


In [6]:
# Define Generator
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()
        self.img_shape = img_shape
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, img_shape[0] * img_shape[1] * img_shape[2]),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *self.img_shape)
        return img

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_shape[0] * img_shape[1] * img_shape[2], 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Define Loss Function
adversarial_loss = nn.BCELoss()

# Initialize Generator and Discriminator
latent_dim = 100
img_shape = (3, 64, 64)  # Change the shape according to your dataset
generator = Generator(latent_dim, img_shape)
discriminator = Discriminator(img_shape)

# GPU Setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)

# Configure Data Loader
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = ImageFolder(root='./abstract_art_gallery', transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Define Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training Loop
num_epochs = 50
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = torch.ones(imgs.size(0), 1, device=device)
        fake = torch.zeros(imgs.size(0), 1, device=device)

        # Configure input
        real_imgs = imgs.to(device)

        # Train Generator
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(imgs.size(0), latent_dim, device=device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]"
        )

[Epoch 0/50] [Batch 0/45] [D loss: 0.6971724033355713] [G loss: 0.6806628704071045]
[Epoch 0/50] [Batch 1/45] [D loss: 0.5902424454689026] [G loss: 0.7796929478645325]
[Epoch 0/50] [Batch 2/45] [D loss: 0.4668735861778259] [G loss: 0.8991663455963135]
[Epoch 0/50] [Batch 3/45] [D loss: 0.32260677218437195] [G loss: 1.0568827390670776]
[Epoch 0/50] [Batch 4/45] [D loss: 0.28790226578712463] [G loss: 1.217278242111206]
[Epoch 0/50] [Batch 5/45] [D loss: 0.2124110460281372] [G loss: 1.3309893608093262]
[Epoch 0/50] [Batch 6/45] [D loss: 0.2052447497844696] [G loss: 1.3547159433364868]
[Epoch 0/50] [Batch 7/45] [D loss: 0.2038901150226593] [G loss: 1.2776672840118408]
[Epoch 0/50] [Batch 8/45] [D loss: 0.24716079235076904] [G loss: 1.1519910097122192]
[Epoch 0/50] [Batch 9/45] [D loss: 0.253895103931427] [G loss: 1.0516223907470703]
[Epoch 0/50] [Batch 10/45] [D loss: 0.2683435380458832] [G loss: 1.0201812982559204]
[Epoch 0/50] [Batch 11/45] [D loss: 0.2740651071071625] [G loss: 1.0179579

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_losses(g_losses, d_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(g_losses, label='Generator Loss')
    plt.plot(d_losses, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('GAN Training Losses')
    plt.legend()
    plt.grid(True)
    plt.show()

# Assuming g_losses and d_losses are lists containing the generator and discriminator losses for each epoch
plot_losses(g_losses, d_losses)

In [ ]:
import torchvision.utils as vutils

# Generate new data
num_samples = 16
z = torch.randn(num_samples, latent_dim, device=device)
with torch.no_grad():
    fake_images = generator(z).detach().cpu()

# Plot the generated images
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Generated Images")
plt.imshow(np.transpose(vutils.make_grid(fake_images, padding=2, normalize=True), (1, 2, 0)))
plt.show()